In [1]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
from scml import pandasx as pdx

In [2]:
percentiles=[.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99]
pd.set_option("use_inf_as_na", True)
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
tqdm.pandas()

In [3]:
txn2id = {
    "clicks": 0,
    "carts": 1,
    "orders": 2,
}
rows = []
with open("input/train.jsonl") as lines:
    for line in tqdm(lines):
        jo = json.loads(line)
        session = int(jo["session"])
        prev = None
        for event in jo["events"]:
            curr = int(event["ts"])
            if prev is not None and curr<prev:
                raise ValueError("event out-of-order")
            rows.append({
                "session": session,
                "aid": int(event["aid"]),
                "ts": curr,
                "type": txn2id[event["type"]]
            })
            prev = curr

12899779it [06:22, 33744.28it/s]


In [4]:
df = pd.DataFrame.from_records(rows)
cols = ["session", "aid"]
df[cols] = df[cols].astype(np.int32)
cols = ["type"]
df[cols] = df[cols].astype(np.int8)
#df.sort_values(["session", "ts"], inplace=True, ignore_index=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216716096 entries, 0 to 216716095
Data columns (total 4 columns):
 #   Column   Dtype
---  ------   -----
 0   session  int32
 1   aid      int32
 2   ts       int64
 3   type     int8 
dtypes: int32(2), int64(1), int8(1)
memory usage: 3.4 GB


In [5]:
df.describe(percentiles=percentiles)

,session,aid,ts,type
count,2.167161e+08,2.167161e+08,2.167161e+08,2.167161e+08
mean,4.702787e+06,9.288052e+05,1.660533e+12,1.250211e-01
std,3.665000e+06,5.366917e+05,6.961450e+08,3.955343e-01
min,0.000000e+00,0.000000e+00,1.659305e+12,0.000000e+00
1%,4.104600e+04,1.763200e+04,1.659349e+12,0.000000e+00
5%,2.118050e+05,9.126300e+04,1.659445e+12,0.000000e+00
10%,4.764980e+05,1.822640e+05,1.659554e+12,0.000000e+00
20%,1.111110e+06,3.716740e+05,1.659811e+12,0.000000e+00
30%,1.866164e+06,5.585730e+05,1.660063e+12,0.000000e+00
40%,2.797679e+06,7.440900e+05,1.660316e+12,0.000000e+00


In [6]:
pdx.value_counts(df["type"])

,count,percent
0,194720954,0.898507
1,16896191,0.077965
2,5098951,0.023528


In [7]:
%%time
assert df.notna().all(axis=None)
df.to_parquet("output/transactions.parquet", index=False)

Wall time: 15 s
